In [12]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2
from knockouts import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
MeanTCounts = pandas.read_csv("./ibdmdb/metaT_per_diagnosis.csv",index_col=0)
NormalizedTCounts = MeanTCounts.div(MeanTCounts.sum())
NormalizedTCounts.head()

,nonIBD,UC,CD,IBD
#FeatureID,,,,
HMPREF9436_RS14285,0.078333,0.039315,0.037492,0.037492
HMPREF9436_RS14685,0.023592,0.008788,0.013451,0.013451
BILO145876EF_RS04385,0.022452,0.003468,0.005717,0.005717
FAEPRAM212_RS14865,0.017201,0.023327,0.030562,0.030562
BSFG_RS23450,0.010633,0.011724,0.010224,0.010224


In [14]:
# condition = "nonIBD"
condition = "IBD"
# condition = "UC"
# condition = "CD"

In [15]:
log_format = '%(asctime)s %(message)s' #%(clientip)-15s %(user)-8s
import logging
from coralme.builder.main import ListHandler
log = logging.getLogger()
logging.basicConfig(filename = "./ibdmdb/integration/integration_{}.log".format(condition), filemode = 'w', level = logging.WARNING, format = log_format)
logging.captureWarnings(True)

In [16]:
survivors = set(pandas.read_csv("survivors.txt",index_col=0,header=None).index.to_list())
done = set(i.split("_killable")[0] for i in os.listdir("./ibdmdb/integration/{}".format(condition)) if "txt" in i)
run_for = survivors #- done
len(run_for)

495

In [18]:
def run(org):
    ListHandler.print_and_log("Loading {}".format(org))
    model = coralme.io.pickle.load_pickle_me_model("./me-models/{}/MEModel-BIO-{}-ME-TS.pkl".format(org,org))
    kill_genes = get_targets(model,condition,NormalizedTCounts)
    index_dct = {r.id : model.reactions.index(r) for r in model.reactions}
    pandas.DataFrame(columns=kill_genes).T.to_csv("./ibdmdb/integration_targets/{}/{}_targets.txt".format(condition,org),header=None)
    
    me_nlp = get_nlp(model)
    del model
    
    f,basis = get_feasibility(me_nlp)
    killable = get_killable(index_dct,me_nlp,basis,kill_genes,limit=lambda x:0,ListHandler=ListHandler,org=org)
    
    pandas.DataFrame(columns=killable).T.to_csv("./ibdmdb/integration/{}/{}_killable.txt".format(condition,org),header=None)

In [ ]:
NP = min([8,len(run_for)])
pool = mp.Pool(NP,maxtasksperchild=1)
pbar = tqdm(total=len(run_for),position=0,leave=True)
pbar.set_description('Building ({} threads)'.format(NP))
def collect_result(result):
    pbar.update(1)

for org in run_for:
    args = ([org])
    pool.apply_async(run,args, callback=collect_result)
pool.close()
pool.join()

Building (8 threads):   0%|          | 0/421 [00:00<?, ?it/s]

Loading Staphylococcus_warneri_SG1Loading Ureaplasma_urealyticum_serovar_8_str_ATCC_27618Loading Bifidobacterium_longum_longum_BBMN68Loading Clostridium_saccharoperbutylacetonicum_N1_4_HMTLoading Bacillus_clausii_KSM_K16Loading Veillonella_parvula_Te3_DSM_2008Loading Collinsella_intestinalis_DSM_13280Loading Citrobacter_youngae_ATCC_29220







Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmpum1329a1.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmpu_4gzore.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmpmewtmran.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Set parameter Username
Academic license - for 

Building (8 threads):   0%|          | 1/421 [01:50<12:52:31, 110.36s/it]

Loading Slackia_exigua_ATCC_700122
Collinsella_intestinalis_DSM_13280 feasible if 521003.7.peg.435 knockout (80 out of 352)
Clostridium_saccharoperbutylacetonicum_N1_4_HMT not feasible if g.32397.CDS.42 knockout (8 out of 1168)
Collinsella_intestinalis_DSM_13280 feasible if 521003.7.peg.436 knockout (81 out of 352)
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmpmauv7vju.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Veillonella_parvula_Te3_DSM_2008 feasible if 479436.15.peg.251 knockout (50 out of 396)
Citrobacter_youngae_ATCC_29220 feasible if 500640.5.peg.64 knockout (17 out of 1108)
Collinsella_intestinalis_DSM_13280 feasible if 521003.7.peg.437 knockout (82 out of 352)
Staphylococcus_warneri_SG1 feasible if g.31042.CDS.129 knockout (36 out of 886)
Bifidobacterium_longum_longum_BBMN68 feasible if 890402.8.peg.457 knockout (36 out of 174)
Veillonella_parvula_Te3_DSM_2008 feasible if 479

Building (8 threads):   0%|          | 2/421 [04:56<18:00:43, 154.76s/it]

...

Loading Ruminococcus_sp_5_1_39BFAA
Lactobacillus_brevis_ATCC_367 not feasible if 387344.13.peg.1964 knockout (537 out of 647)
Streptococcus_sanguinis_SK36 not feasible if g.1182.peg.1699 knockout (556 out of 754)
Marvinbryantia_formatexigens_I_52_DSM_14469 feasible if 478749.5.peg.2270 knockout (310 out of 589)
Lactobacillus_reuteri_SD2112_ATCC_55730 not feasible if 491077.3.peg.640 knockout (185 out of 656)
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmpp15qbcqc.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Marvinbryantia_formatexigens_I_52_DSM_14469 feasible if 478749.5.peg.2356 knockout (311 out of 589)
Escherichia_fergusonii_ATCC_35469 feasible if g.2499.peg.1824 knockout (272 out of 628)
Lactobacillus_brevis_ATCC_367 feasible if 387344.13.peg.1993 knockout (538 out of 647)
Marvinbryantia_formatexigens_I_52_DSM_14469 feasible if 478749.5.peg.2357 knockout (312 out of 589)
Lactobacill

Building (8 threads):   5%|▍         | 19/421 [1:10:51<24:18:49, 217.74s/it]

Lactobacillus_reuteri_SD2112_ATCC_55730 feasible if 491077.3.peg.772 knockout (247 out of 656)
Loading Campylobacter_upsaliensis_JV21
Escherichia_fergusonii_ATCC_35469 feasible if g.2499.peg.1993 knockout (286 out of 628)
Marvinbryantia_formatexigens_I_52_DSM_14469 feasible if 478749.5.peg.2675 knockout (368 out of 589)
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmp3s0c7ky4.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Lactobacillus_reuteri_SD2112_ATCC_55730 not feasible if 491077.3.peg.773 knockout (248 out of 656)
Micromonospora_aurantiaca_ATCC_27029 feasible if 644283.3.peg.4350 knockout (725 out of 1160)
Streptococcus_sanguinis_SK36 feasible if g.1182.peg.1882 knockout (593 out of 754)
Lactobacillus_reuteri_SD2112_ATCC_55730 not feasible if 491077.3.peg.774 knockout (249 out of 656)
Marvinbryantia_formatexigens_I_52_DSM_14469 not feasible if 478749.5.peg.2686 knockout (369 out of 58

Building (8 threads):   5%|▍         | 20/421 [1:14:47<24:52:31, 223.32s/it]

Loading Corynebacterium_durum_F0235
Bacillus_clausii_KSM_K16 not feasible if g.2880.peg.4235 knockout (1073 out of 1174)
Escherichia_fergusonii_ATCC_35469 feasible if g.2499.peg.2311 knockout (333 out of 628)
Campylobacter_upsaliensis_JV21 not feasible if 888826.3.peg.694 knockout (201 out of 505)
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmp1yn0vm9g.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Campylobacter_upsaliensis_JV21 not feasible if 888826.3.peg.695 knockout (202 out of 505)
Marvinbryantia_formatexigens_I_52_DSM_14469 feasible if 478749.5.peg.3877 knockout (506 out of 589)
Streptococcus_sanguinis_SK36 not feasible if g.1182.peg.2118 knockout (664 out of 754)
Campylobacter_upsaliensis_JV21 not feasible if 888826.3.peg.697 knockout (203 out of 505)
Campylobacter_upsaliensis_JV21 not feasible if 888826.3.peg.699 knockout (204 out of 505)
Micromonospora_aurantiaca_ATCC_27029 not f

In [21]:
2

2

In [ ]:
# l1 = set("RNA_"+i for i in pandas.read_csv("./ibdmdb/integration/nonIBD/Abiotrophia_defectiva_ATCC_49176_killable.txt",index_col=0).index)
# l2 = set(pandas.read_csv("./ibdmdb/integration/nonIBD/_Abiotrophia_defectiva_ATCC_49176_killable.txt",index_col=0).index)
# l3 = set("RNA_"+i for i in pandas.read_csv("./ibdmdb/reduction/nonIBD/Abiotrophia_defectiva_ATCC_49176_killable.txt",index_col=0).index)